<a href="https://colab.research.google.com/github/hannape/Gruba-kreska/blob/main/2_Wczytanie_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Będzie to funkcja która pozwoli wczytać dane z pliku wav, podzieli je na M-ms klipy (500 ms domyślne) z N-sek overlapem (150 ms), wczyta etykiety z pliku txt (jeśli więcej niż 4ms głosu w próbce - to pozytywna), stworzy wektory 

* X_train i Y_train
* X_valid i Y_valid

W sumie mam 94 nagrania. 

* treningowy - 87 nagrań, balansujemy, jeśli są dostępne inne etykiety niż interesujących nas głosów (np gh, t, g), to dodajemy je jako próbki negatywne. Jeśłi nie ma głosów w nagraniu to bierzemy P (50) próbek losowych z tego nagrania. 
* walidacyjny to 7-8 nagrań, które balansujemy tylko trochę - np z poziomu 1:100, do poziomu 1:10.



**Ale najpierw, trzeba jakoś wybrać te nagrania do walida. trzeba sprawdzić stosunki s : k głównie, może jakoś uwzględnić d i r, choć ich dużo mniej, więc z inną wagą**





In [1]:
# Wczytywanie google drive'a
from google.colab import drive
drive.mount('/content/drive')
from __future__ import print_function

Mounted at /content/drive


In [2]:
import os
import contextlib
import numpy as np
import wave
import math
import pandas as pd

In [3]:
# Parametry które będą wchodzić do funkcji

path_test1618_txt = 'drive/My Drive/poprawione etykiety 24112020/testowe/'
path_train161718_txt =  'drive/My Drive/poprawione etykiety 24112020/treningowe/'
path_test1618_wav = 'drive/My Drive/testowe_1618/'
path_train161718_wav =  'drive/My Drive/treningowe94_161718/'
balance_type = ['balanced', 'valid', 'full_rec']  # na potrzeby treningu, valida, testu
chunk_length_ms = 500 # 500 ms
chunk_overlap = 150 # 150 ms
calls_0 = ['t', 'g', 'czapla', 'gh', 'puszczyk']                                    # etykiety które są negatywne
calls_1 = ['d', 'd?', 'k', 'k?', 'kwiczol', 'r','r?', 's', 's?', 'skowronek', 'ni'] # calls of interest
calls_unknown = ['???','??? mysz', '??? high freq']                                 # unknown, nie wiem czy to głos czy nie, więc będę wyrzucać te chunksy to zawierające
tolerance = 0.004  # jaka tolerancja, w sek (jeśli mniej niż ta wartość znajdzie się w chunksie, to uznajemy że głosu tam nie ma)

In [4]:
def my_read_labels(label_name,path_txt):  # ta sama funkcja co w 1. poprawianie danych
    '''Wczytuje etykiety czasowe z pliku labels.txt w folderze train.
    
    Returns:
        ndarray: Tablica z etykietami czasowymi zawierająca kolumny:  sekunda początku dźwięku, sekunda końca dźwięku, gatunek.
    '''
    labels = []
    with open(os.path.join(path_txt, label_name + '.txt'), 'r') as file:
        text = file.read()
        for line in text.split('\n')[0:]:
            if len(line) > 1:
                start, stop, spec = line.split('\t')
                #print(start)
                start, stop, spec = float(start), float(stop), str(spec),
                labels.append([start, stop, spec])
    return np.array(labels)


def my_check_labels(second, chunk_length_s, labels, tol=0.):
    '''Sprawdza czy w ramce czasowej [second, second + chunk_length_s] znajduje się coś wg etykiet `labels`.
    
    Args:
        second (float): Sekunda nagrania.
        labels (ndarray): Tablica z etykietami, której 1 kolumna oznacza początek, a 2ga - koniec nagrania.
        tol (float): Tolerancja na brzegach fragmentu. Dźwięk, żeby był uznany, musi się kończyć po czasie `second+tol`
            lub zaczynać przed czasem `second+chunk_length-tol`.
    Returns:
        bool: Czy w ramce czasowej jest etykieta - co najmniej 4 ms labelu.
 ''' 
    
  #calls_to_cut = ['t', 't?', 't ','t  ', 'g', 'czapla', 'gh', 'g cz', 'puszczyk','gaski','g?','mewa?','zwierzak?','high freq','g niskie','??? mysz']

  # jeśli początek etykiety zawiera się w chunku, najpóźniej 4ms przed końcem chunka
  # jeśli koniec etykiety zawiera się w chunku, i najwcześniej 4 ms po rozpoczęciu chunka
  # jeśli początek etykiety zaczyna się rpzed, a koniec po (w przypadku głosów dłuższych niż chunk length, b. rzadkie przypadki dla 0.5 sek)
    return (float(labels[0]) >= second and float(labels[0]) < second + chunk_length_s - tol) or \
           (float(labels[1]) < second + chunk_length_s and float(labels[1]) > second + tol) or \
           (float(labels[0]) < second and float(labels[1]) > second + chunk_length_s)              
           #and (labels[2] in calls_to_cut)    # nie sprawdzajmy tu jaka to etykieta jest

def my_map_seconds_to_y(labels, recording_duration, calls_of_interest, calls_to_cut, calls_unknown):  ## Chcemy 500 ms z 150 ms overlapem
    '''Tworzy etykiety dla każdego kolejnego chunksa. 1 gdy co najmniej 4 ms etykiety się znajdują w chunksie (gdy urwane dźwięki na brzegach, <4ms - to 0).
    
    Args:
        labels (ndarray): Tablica z etykietami, której 1 kolumna oznacza początek, a druga - koniec nagrania.
    Returns:
        ndarray: Tablica z binarnymi etykietami.
    '''
    #STARE: calls_to_cut = ['t', 't?', 't ','t  ', 'g', 'czapla', 'gh', 'g cz', 'puszczyk','gaski','g?','mewa?','zwierzak?','high freq','g niskie','??? mysz']
    
    
    duration_in_ms = recording_duration*1000
    nr_of_chunks =  1 + (duration_in_ms - chunk_length_ms) / (chunk_length_ms - chunk_overlap)
    print("ilość chunksów w tym nagraniu: ", math.ceil(nr_of_chunks) )
    
    
    y = [0] * math.ceil(nr_of_chunks)             # recording_duration 
    y_restrictive = [0] * math.ceil(nr_of_chunks) # recording_duration
    chunks_start = [0] * math.ceil(nr_of_chunks)     
    chunks_end = [0] * math.ceil(nr_of_chunks) 
    has_unknown = [0] * math.ceil(nr_of_chunks)
    call_id = [[] for _ in range(math.ceil(nr_of_chunks))]
    chunks_species = [[] for _ in range(math.ceil(nr_of_chunks))]
    
    print(np.shape(labels))
    for s in range(math.ceil(nr_of_chunks)):
        chunks_start[s] = s * ((chunk_length_ms-chunk_overlap) / 1000)  # czyli s * 0.35
        chunks_end[s] = chunks_start[s] + chunk_length_ms/1000 
        for ind,l in enumerate(labels):
            #print('label_index: ',ind, ' a label to: ', l[2] )
            if my_check_labels(chunks_start[s], chunk_length_ms/1000, l):
                #y[s] = 1
                #print(s*0.35)
                if l[2] in calls_to_cut:
                  y[s] = 0
                if l[2] in calls_unknown:
                  y[s] = 0 
                  has_unknown[s] = 1
                if l[2] in calls_of_interest:  # ostatnie, bo jeśli jest więcej niż 1 etykieta, to najważniejsze jest że w chunksie jest też głos
                  y[s] = 1 
                    
                  #chunks_species[s]= l[2]  
                  #call_id[s] = ind[2] 

            if my_check_labels(chunks_start[s], chunk_length_ms/1000, l, 0.004): #z tolerancją 
                  #y_restrictive[s] = 0
                  #print("wyciety glos:")
                  #print(l[2])
                if l[2] in calls_to_cut:
                  y_restrictive[s] = 0
                if l[2] in calls_unknown:
                  y_restrictive[s] = 0  
                  has_unknown[s] = 1
                if l[2] in calls_of_interest:  # ostatnie, bo jeśli jest więcej niż 1 etykieta, to najważniejsze jest że w chunksie jest też głos
                  
                  #if y_restrictive[s] == 0:  # pierwszy label 
                  y_restrictive[s] = 1  
                  chunks_species[s].append(l[2])   # może być w tym, nie w restrictive - by było wiadomo jaki typ głosu zostanie wycięty
                  call_id[s].append(ind)
              
                      
        if y[s] != y_restrictive[s] and l[2] in calls_of_interest:
            print('próbka ', s, 'zaczynajaca sie ', chunks_start[s],', jest unknown, bo za mały fragment głosu ')
            #print('y[s]:', y[s],'yres:', y_restrictive[s] )
            y[s] = 0    # jeśli mniej niż 4 ms fragment, to zakładamy że nie ma głosu, i dajemy unknown także, by nie karać za ewentualne znalezienie tego głosu
            has_unknown[s] = 1
    print('Rozmiar ',np.shape([s, chunks_start, chunks_end, y, chunks_species, call_id, has_unknown]))
    return s+1, chunks_start, chunks_end, y, chunks_species, call_id, has_unknown    


def my_load_wav(path_wav, path_txt):   # dłuższa funkcja niż w 1.

  recording_labels_all = []
  in_which_recording = []
  file_names = []

  rec_files = [file_name for file_name in os.listdir(path_wav) if file_name.endswith('.wav')]

  print(np.size(rec_files))

  X_matrix = [[] for _ in range(np.size(rec_files))]
  file_names = [[] for _ in range(np.size(rec_files))]

  for ind, file_name in enumerate(rec_files):
      
      print("------------Analiza nagrania: " + file_name + "-----------")
      recording_id = str(file_name.split('.')[0])
      
      fname = path_wav + file_name      #print(fname)
      with contextlib.closing(wave.open(fname,'r')) as f:
          frames = f.getnframes()
          rate = f.getframerate()
          duration = frames / float(rate)
          recording_duration = math.ceil(duration)
          print("Czas trwania nagrania - w sekundach: " + str(recording_duration))
          #print("rate:", rate)
    
      recording_id = (file_name.split('.')[0])       #print(recording_id)
      recording_labels = my_read_labels(recording_id, path_txt)    

      print("Duration:", duration)
      #y_binary = my_map_seconds_to_y(recording_labels, duration)
      chunk_id, chunk_start, chunk_end, has_bird, chunks_species, call_id, has_unknown  = my_map_seconds_to_y(recording_labels, duration, calls_1, calls_0, calls_unknown)
      chunk_ids = range(chunk_id)
      chunk_start = rate*np.array(chunk_start)
      chunk_start = [round(num) for num in chunk_start]
      chunk_end = rate*np.array(chunk_end)
      chunk_end = [round(num) for num in chunk_end]
      X_matrix[ind] = [chunk_ids, chunk_start, chunk_end, has_bird, chunks_species, call_id, has_unknown]  # !!!! chunk start i end  przerzucone na sample 1!!!!!
      file_names[ind] = file_name

  return X_matrix , file_names #chunk_id, chunk_start, chunk_end, has_bird, chunks_species, call_id, has_unknown

In [ ]:
#chunk_id, chunk_start, chunk_end, has_bird, chunks_species, call_id, has_unknown = my_load_wav(path_train161718_wav, path_train161718_txt) 
X_train, file_names_train = my_load_wav(path_train161718_wav, path_train161718_txt) 

In [ ]:
print(np.shape(X_train))
czas_trwania_spr= np.array(X_train[0][1])- np.array(X_train[0][2])
print(np.unique(czas_trwania_spr)) # upewnienie się że wszystkie chunki jednakowej długości są
print(np.shape(file_names_train))

result_dataframe = pd.DataFrame(data = X_train, index=file_names_train, columns = ['chunk_ids', 'chunk_start', 'chunk_end', 'has_bird', 'chunks_species', 'call_id', 'has_unknown'])

In [ ]:
X_test, file_names_test = my_load_wav(path_test1618_wav, path_test1618_txt) 


In [ ]:
#@title
a = (X_train[17][1])
print(type(X_train))
print(type(result_dataframe['chunk_start']))
print(result_dataframe['chunk_start']["BUK4_20160922_005604.wav"])

In [ ]:
#@title
# do sprawdzenia czy te zagnieżdżone listy w call_id i chunks_species działają

def isListEmpty(inList):
  for elem in inList:
    if elem: # Is a list
        print(elem)
        #return all( map(isListEmpty, inList) )
    #return False # Not a list

isListEmpty(result_dataframe['call_id'][2])
isListEmpty(result_dataframe['chunks_species'][2])


In [ ]:
#@title
# do sprawdzenia jakie ID mają głosy w naszym nagraniu (każdy z labeli ma ID, nie tylko głosy, dlatego nie wszystkie liczby tu będą)

test_list = result_dataframe['call_id'][2]

#print("The original list : " + str(test_list)) 
  
res = [] 
temp = set() 
for inner in test_list: 
        for ele in inner: 
            if not ele in temp: 
                temp.add(ele) 
                res.append(ele) 
  
print("Unique elements in nested tuples are : " + str(res)) 

In [50]:
rec_for_train = [ 181006, 181007, 181001, 181002, 161103, 161104, 160925, 160926, 161031, 161005, 160918 ] 
labels_s = ['s', 's?']
labels_k = ['k', 'k?']
labels_d = ['d', 'd?']
labels_r = ['r', 'r?']
calls_1 = ['d', 'd?', 'k', 'k?', 'kwiczol', 'r','r?', 's', 's?', 'skowronek', 'ni']

In [81]:
# Podstawowy wybór nagrań treningowych i walidacyjnych praz zbiór testowy: stare 20 nagrań, nowe 18 nagrań

all_rec_dates_train, all_rec_dates_test = [],[]
potential_valid_rec = []
basic_train_rec = []
test_new_rec,test_old_rec = [], []

for file_name in file_names_train:

  recording_date = str(file_name.split('_20')[1][0:6])
  all_rec_dates_train.append(int(recording_date))
  if int(recording_date) not in rec_for_train:
    potential_valid_rec.append(file_name.split('.')[0])
  else:
    basic_train_rec.append(file_name.split('.')[0]) 
    
print(all_rec_dates_train)
print(potential_valid_rec)
print(basic_train_rec)

for file_name in file_names_test:

  recording_date = str(file_name.split('_20')[1][0:6])
  all_rec_dates_test.append(int(recording_date))
  test_old_rec.append(file_name.split('.')[0])
  if int(recording_date) != 161101:
    test_new_rec.append(file_name.split('.')[0])

print(test_new_rec)
print(test_old_rec)

[161031, 161104, 160918, 161024, 160925, 161104, 161104, 161104, 161019, 160926, 161103, 161104, 161104, 161005, 160925, 160925, 161006, 161002, 160926, 161025, 161104, 161031, 160926, 161103, 160926, 160926, 161104, 160926, 161022, 160925, 161103, 161005, 161013, 160926, 160925, 161104, 160925, 160919, 160918, 161104, 161015, 160925, 161011, 161104, 161104, 160914, 160925, 160926, 160925, 160925, 170920, 171022, 170902, 171026, 171001, 170906, 171007, 170904, 170910, 170920, 171007, 171017, 170923, 170912, 171102, 181004, 181029, 181002, 181007, 181006, 181007, 181007, 181006, 181007, 181007, 181007, 181007, 181006, 181006, 181007, 181003, 181006, 181006, 181006, 181001, 181007, 181006, 180930, 181009, 181006, 181007, 181007, 181002, 181006]
['BUK4_20161024_223604', 'BUK4_20161019_011904', '7wiatr_BUK4_20161006_002104', 'BUK4_20161002_235805', '9niski_szum_BUK4_20161025_000604', 'BUK4_20161022_224004', 'BUK4_20161013_200104', 'BUK4_20160919_013304', '8deszczk_bezptasio_BUK4_20161015_2

In [97]:
def find_ratio(recordings_names, path_txt):
  count_s,count_k, count_d, count_r, count_positives = 0,0,0,0,0
  for rec_name in recordings_names:
    recording_labels = my_read_labels(rec_name, path_txt)  
    #print(recording_labels)
    counted_labels = Counter(recording_labels[:,2])

    #print(counted_labels)
    for calls in labels_s:
      count_s += counted_labels[calls]

    for calls in labels_k:
      count_k += counted_labels[calls]

    for calls in labels_d:
      count_d += counted_labels[calls]   

    for calls in labels_r:
      count_r += counted_labels[calls] 

    for calls in calls_1:
      count_positives += counted_labels[calls] 

  return [count_s, count_k, count_d, count_r, count_positives]

count_ratio_valid = find_ratio(potential_valid_rec, path_train161718_txt)  
count_ratio_train = find_ratio(basic_train_rec, path_train161718_txt)
count_ratio_test_old= find_ratio(test_old_rec, path_test1618_txt)  # wszystkie 20 nagrań
count_ratio_test_new= find_ratio(test_new_rec, path_test1618_txt)  # bez tych 2 zawierających dużo k, czyli 18 nagrań

print('Ratio s : k : d : r')
print('Valid (31): ', count_ratio_valid[0],' : ',count_ratio_valid[1],' : ',count_ratio_valid[2],' : ',count_ratio_valid[3],'. Ratio s:k - ', count_ratio_valid[0]/count_ratio_valid[1])  
print('Train (63): ', count_ratio_train[0],' : ',count_ratio_train[1],' : ',count_ratio_train[2],' : ',count_ratio_train[3], '. Ratio s:k - ', count_ratio_train[0]/count_ratio_train[1])  
print('Test old (20): ', count_ratio_test_old[0],' : ',count_ratio_test_old[1],' : ',count_ratio_test_old[2],' : ',count_ratio_test_old[3], '. Ratio s:k - ', count_ratio_test_old[0]/count_ratio_test_old[1])  
print('Test new (18): ',count_ratio_test_new[0],' : ',count_ratio_test_new[1],' : ',count_ratio_test_new[2],' : ',count_ratio_test_new[3], '. Ratio s:k - ', count_ratio_test_new[0]/count_ratio_test_new[1])  

#print(np.shape(potential_valid_dates))
#print(np.shape(basic_train_rec))

Ratio s : k : d : r
Valid (31):  519  :  77  :  49  :  20 . Ratio s:k -  6.740259740259741
Train (63):  3133  :  597  :  119  :  69 . Ratio s:k -  5.247906197654942
Test old (20):  295  :  399  :  20  :  19 . Ratio s:k -  0.7393483709273183
Test new (18):  285  :  75  :  11  :  19 . Ratio s:k -  3.8


In [158]:
import random

## funkcja losująca 8 nagrań do walida (z 31), resztę wrzucająca do traina, przeliczenie score
# scorer = stosunek s:k najważniejszy, a tak z 1/5 wagi na d
scorer_best, scorer = 10, 10

for i in range(1000):

  random_valid_rec = random.sample(potential_valid_rec, 8)
  remaining_train_rec = []


  #print(random_valid_rec)

  for file_name in file_names_train:
    #print(file_name)
    if (file_name.split('.')[0]) not in random_valid_rec:
    #print(file_name)
      remaining_train_rec.append(file_name.split('.')[0])

    
  count_ratio_valid_8 = find_ratio(random_valid_rec, path_train161718_txt)
  count_ratio_train63_23 = find_ratio(remaining_train_rec, path_train161718_txt)

  #print(np.shape(remaining_train_rec))
  #print(np.shape(random_valid_rec))
  #print((remaining_train_rec))
  #print((random_valid_rec))
  if (count_ratio_valid_8[2]!=0):
    scorer_kd = 0.2*abs((count_ratio_train63_23[1]/count_ratio_train63_23[2]) - ( count_ratio_valid_8[1]/count_ratio_valid_8[2] ))
  else:
    scorer_kd = 666

  if (count_ratio_valid_8[1]!=0):
    scorer_sk = abs((count_ratio_train63_23[0]/count_ratio_train63_23[1]) - ( count_ratio_valid_8[0]/count_ratio_valid_8[1] ) )
  else:
    scorer_sk = 1000000  

  scorer = scorer_sk + scorer_kd

  print(scorer)
  if scorer_best > scorer:
    print(i)
    print('scorer: ', scorer)
    print('Valid ratio s:k - ', count_ratio_valid_8[0]/count_ratio_valid_8[1])
    print('Train ratio s:k - ', count_ratio_train63_23[0]/count_ratio_train63_23[1])
    print('Valid ratio k:d - ', count_ratio_valid_8[1]/count_ratio_valid_8[2])
    print('Train ratio k:d - ', count_ratio_train63_23[1]/count_ratio_train63_23[2])
    scorer_best =  scorer
    validate_rec_best = random_valid_rec
    count_ratio_valid_best = count_ratio_valid_8
    count_ratio_train_best = count_ratio_train63_23




3.952229504241889
0
scorer:  3.952229504241889
Valid ratio s:k -  8.785714285714286
Train ratio s:k -  5.272445820433436
Valid ratio k:d -  2.0
Train ratio k:d -  4.194805194805195
2.407146174259177
1
scorer:  2.407146174259177
Valid ratio s:k -  3.2142857142857144
Train ratio s:k -  5.513931888544891
Valid ratio k:d -  3.5
Train ratio k:d -  4.0375
1000000.8120481927
8.80328543796629
107.0069599827883
25.848032849962724
4.778521229996753
3.0790808257995934
17.087825930588025
3.7054756608886836
14.563674412284275
8.695338239184284
3.917566330321748
2.9726879764450542
8.212013083006873
2.795372670807454
1.3849430199430204
16
scorer:  1.3849430199430204
Valid ratio s:k -  6.038461538461538
Train ratio s:k -  5.393518518518518
Valid ratio k:d -  0.9285714285714286
Train ratio k:d -  4.628571428571429
3.065512469197909
3.480860805860806
2.6761696260517223
1.8560132421578202
4.242262299347375
7.06417914597079
1.3010464185456057
23
scorer:  1.3010464185456057
Valid ratio s:k -  5.96774193548

In [ ]:
print(validate_rec_best)
print('Valid (8): ', count_ratio_valid_best[0],' : ',count_ratio_valid_best[1],' : ',count_ratio_valid_best[2],' : ',count_ratio_valid_best[3],'. Ratio s:k - ', count_ratio_valid_best[0]/count_ratio_valid_best[1], 'Ratio k:d - ', count_ratio_valid_best[1]/count_ratio_valid_best[2], 'Ratio d:r - ', count_ratio_valid_best[2]/count_ratio_valid_best[3])  
print('Train (86): ', count_ratio_train_best[0],' : ',count_ratio_train_best[1],' : ',count_ratio_train_best[2],' : ',count_ratio_train_best[3], '. Ratio s:k - ', count_ratio_train_best[0]/count_ratio_train_best[1], 'Ratio k:d - ', count_ratio_train_best[1]/count_ratio_train_best[2], 'Ratio d:r - ', count_ratio_train_best[2]/count_ratio_train_best[3])  

## Upewnić się że się daty nie pokrywają
validate_rec_best_dates = []
rec_dates_train, rec_dates_valid = [],[]

for file_name in validate_rec_best:
  recording_date = str(file_name.split('_20')[1][0:6])
  validate_rec_best_dates.append(recording_date)

print(validate_rec_best_dates)

for file_name in file_names_train:
  #print(file_name)
  recording_date = str(file_name.split('_20')[1][0:6])
  #print(recording_date)
  if (recording_date) in validate_rec_best_dates:
    rec_dates_valid.append(recording_date)
  else:  # final trening 88
    rec_dates_train.append(recording_date) 
# Nie pokrywają się

#Counter(rec_dates_valid)
#Counter(rec_dates_train)

In [ ]:
### TO JEST OD DZIŚ MÓJ WALID

valid_set = ['9niski_szum_BUK4_20161025_000604', 'BUK5_20180930_000704', 'BUK4_20161024_223604', 'BUK4_20171001_020404a', 'BUK1_20160914_011604', 'BUK5_20170910_025605', 'BUK4_20161013_200104', 'BUK5_20181003_235705']
